In [ ]:
from google.colab import drive  # 구글 코랩에서 구글 드라이브(파일)접근을 위한 라이브러리 import
drive.mount('/content/gdrive')  # 구글 드라이브 mount

# 리눅스 명령어를 사용해서 실습파일 'ChatbotData.csv'이 있는 폴더로 이동
%cd "/content/gdrive/MyDrive/고려사이버대학교/2024-1 AI개발실무"
# 리눅스 명령어를 사용해서 폴더내 파일 확인
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/고려사이버대학교/2024-1 AI개발실무
'2024-1 AI 개발 실무 14주차 리포트.ipynb'   ChatbotData.csv


In [ ]:
# 기존 예제 코드 분석

# 문장 또는 문서의 TF-IDF 가중치를 계산하는 함수 import
from sklearn.feature_extraction.text import TfidfVectorizer
# 두 벡터 간의 코사인 유사도를 계산하는 함수 import
from sklearn.metrics.pairwise import cosine_similarity

# TfidfVectorizer 객체 생성
vectorizer = TfidfVectorizer()

# 한국어 문장 작성
sentence1 = "저는 오늘 밥을 먹었습니다."
sentence2 = "저는 어제 밥을 먹었습니다."
# sentence2 = "너는 그제 밥을 지었습니다."

# sentence1 = "나는 학교에 갔다"
# sentence2 = "나는 영화관에 갔다"

# 두 문장을 TF-IDF 벡터로 변환(문장 내 각 단어의 중요도를 수치화)
tfidf_matrix = vectorizer.fit_transform([sentence1, sentence2]) # Scipy의 CSR(Compressed Sparse Row) 형식의 희소 행렬(sparse matrix), 이 형식은 2차원 배열의 데이터를 메모리 효율적으로 저장하기 위해 0이 아닌 값들만을 저장

# print( tfidf_matrix.toarray())

# 두 문장의 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2]) # tfidf_matrix[0:1]는 첫 번째 행을 선택하는 슬라이싱
print(cosine_sim, type(cosine_sim))

print(f"문장 1: {sentence1}")
print(f"문장 2: {sentence2}")
print(f"두 문장의 코사인 유사도: {cosine_sim[0][0]}")


[[0.60297482]] <class 'numpy.ndarray'>
문장 1: 저는 오늘 밥을 먹었습니다.
문장 2: 저는 어제 밥을 먹었습니다.
두 문장의 코사인 유사도: 0.6029748160380572


In [ ]:
# 기존 챗봇 예제 코드 분석

# 파이썬 데이터 분석 라이브러리인 판다스 improt
import pandas as pd

# sklearn라는 머신러닝 라이브러리에서 TfidfVectorizer와 cosine_similarity를 불러옴
# TfidfVectorizer는 문서의 텍스트 데이터를 벡터 형태로 변환하는데 사용하며, cosine_similarity는 두 벡터 간의 코사인 유사도를 계산
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 챗봇 클래스를 정의
class SimpleChatBot:
    # 챗봇 객체를 초기화하는 메서드, 초기화 시에는 입력된 데이터 파일을 로드하고, TfidfVectorizer를 사용해 질문 데이터를 벡터화함
    def __init__(self, filepath):
        self.questions, self.answers = self.load_data(filepath) ## 데이터 파일 로드
        self.vectorizer = TfidfVectorizer()
        self.question_vectors = self.vectorizer.fit_transform(self.questions)  # 질문을 TF-IDF로 변환

    # CSV 파일로부터 질문과 답변 데이터를 불러오는 메서드
    def load_data(self, filepath):
        data = pd.read_csv(filepath)
        questions = data['Q'].tolist()  # 질문열만 뽑아 파이썬 리스트로 저장
        answers = data['A'].tolist()  # 답변열만 뽑아 파이썬 리스트로 저장
        return questions, answers

    # 입력 문장에 가장 잘 맞는 답변을 찾는 메서드, 입력 문장을 벡터화하고, 이를 기존 질문 벡터들과 비교하여 가장 높은 유사도를 가진 질문의 답변을 반환함
    def find_best_answer(self, input_sentence):
        # 사용자 입력 문장을 벡터화
        input_vector = self.vectorizer.transform([input_sentence])
        # 사용자 입력 벡터와 기존 질문 벡터들 간의 코사인 유사도를 계산
        similarities = cosine_similarity(input_vector, self.question_vectors) # 코사인 유사도 값들을 저장

        # 가장 유사도가 높은 질문의 인덱스를 찾음
        best_match_index = similarities.argmax()   # 유사도 값이 가장 큰 값의 인덱스를 반환
        # 가장 유사한 질문에 해당하는 답변을 반환
        return self.answers[best_match_index]

# 데이터 파일의 경로를 지정합니다.
filepath = 'ChatbotData.csv'

# 챗봇 객체를 생성합니다.
chatbot = SimpleChatBot(filepath)

# '종료'라는 입력이 나올 때까지 사용자의 입력에 따라 챗봇의 응답을 출력하는 무한 루프를 실행합니다.
while True:
    input_sentence = input('You: ')
    if input_sentence.lower() == '종료':
        break
    response = chatbot.find_best_answer(input_sentence)
    print('Chatbot:', response)

You: 안녕
Chatbot: 안녕하세요.
You: 종료


In [ ]:
import pandas as pd
import numpy as np

# 레벤슈타인 거리 계산 함수 작성
# 레벤슈타인 거리는, 두 문자열 간의 편집 거리를 측정하여 문자열 간의 유사성을 측정하는 유사도 분석 알고리즘
def levenshtein_distance(sentence1, sentence2):
    if sentence1 == sentence2: # sentence1와 sentence2가 같은 문장이면 레벤슈타인 거리는 0
        return 0

    if len(sentence1) < len(sentence2): # 길이가 긴 문장을 sentence1으로 변경
        sentence1, sentence2 = (sentence2, sentence1)

    if len(sentence2) == 0: # sentence2의 길이가 0이면, 레벤슈타인 거리는 sentence1의 길이
        return len(sentence1)

    previous_row = range(len(sentence2) + 1)  # sentence2에 대한 0열 초기화
    matrix = [[0 if i else j for j in range(len(sentence2) + 1)] for i in range(len(sentence1) + 1)] # (len(sentence2) + 1) * (len(sentence1) + 1)
    # print(len(sentence2), len(sentence1))
    # print(len(matrix), len(matrix[0]))

    for i, c1 in enumerate(sentence1):  # i는 sentence1의 인덱스, c는 sentence1의 값이 순차적으로 들어감
        matrix[i+1][0] = current_row = i + 1 # i는 0부터 시작하고, row는 1부터 시작하기 때문에 1을 더해줌

        # sentence1의 한글자 c에 대해 sentence2의 길이만큼 반복
        for j, c2 in enumerate(sentence2):  # i는 sentence1의 인덱스, c2는 sentence2의 값이 순차적으로 들어감
            # 삽입이 필요한 경우는 , 왼쪽 수에서 +1
            insertions = matrix[current_row-1][j + 1] + 1
            # 삭제가 필요한 경우는, 위쪽 수에서 +1
            deletions = matrix[current_row][j] + 1
            # 수정이 필요한 경우는, 왼쪽위(대각선) 수에서 +1, 글자가 동일하면 +0
            substitutions = matrix[current_row-1][j] + int(c1 != c2)

            # insertions, deletions, substitutions 중에서 가장 작은 값을 저장
            matrix[i+1][j + 1] = min(insertions, deletions, substitutions)
            # print(f"{insertions =} {deletions = } {substitutions = }")

        previous_row = current_row
    for i in matrix:
        print(i)
    return matrix[-1][-1] # 최종적으로 마지막 수가 결과

# 강의에서 사용한 예제로 테스트
sentence1 = "유사도나 분석 할까요"
sentence2 = "얼마나 분석이 될까요"
levenshtein_distance(sentence1, sentence2)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[2, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[3, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[4, 4, 4, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[5, 5, 5, 4, 3, 4, 5, 6, 7, 8, 9, 10]
[6, 6, 6, 5, 4, 3, 4, 5, 6, 7, 8, 9]
[7, 7, 7, 6, 5, 4, 3, 4, 5, 6, 7, 8]
[8, 8, 8, 7, 6, 5, 4, 4, 4, 5, 6, 7]
[9, 9, 9, 8, 7, 6, 5, 5, 5, 5, 6, 7]
[10, 10, 10, 9, 8, 7, 6, 6, 6, 6, 5, 6]
[11, 11, 11, 10, 9, 8, 7, 7, 7, 7, 6, 5]


5

In [ ]:
import pandas as pd
import numpy as np

# 레벤슈타인 거리 계산 함수 작성
# 레벤슈타인 거리는, 두 문자열 간의 편집 거리를 측정하여 문자열 간의 유사성을 측정하는 유사도 분석 알고리즘
def levenshtein_distance(sentence1, sentence2):
    if sentence1 == sentence2: # sentence1와 sentence2가 같은 문장이면 레벤슈타인 거리는 0
        return 0

    if len(sentence1) < len(sentence2): # 길이가 긴 문장을 sentence1으로 변경
        sentence1, sentence2 = (sentence2, sentence1)

    if len(sentence2) == 0: # sentence2의 길이가 0이면, 레벤슈타인 거리는 sentence1의 길이
        return len(sentence1)

    previous_row = range(len(sentence2) + 1)  # sentence2에 대한 0열 초기화
    matrix = [[0 if i else j for j in range(len(sentence2) + 1)] for i in range(len(sentence1) + 1)] # (len(sentence2) + 1) * (len(sentence1) + 1)
    # print(len(sentence2), len(sentence1))
    # print(len(matrix), len(matrix[0]))

    for i, c1 in enumerate(sentence1):  # i는 sentence1의 인덱스, c는 sentence1의 값이 순차적으로 들어감
        matrix[i+1][0] = current_row = i + 1 # i는 0부터 시작하고, row는 1부터 시작하기 때문에 1을 더해줌

        # sentence1의 한글자 c에 대해 sentence2의 길이만큼 반복
        for j, c2 in enumerate(sentence2):  # i는 sentence1의 인덱스, c2는 sentence2의 값이 순차적으로 들어감
            # 삽입이 필요한 경우는 , 왼쪽 수에서 +1
            insertions = matrix[current_row-1][j + 1] + 1
            # 삭제가 필요한 경우는, 위쪽 수에서 +1
            deletions = matrix[current_row][j] + 1
            # 수정이 필요한 경우는, 왼쪽위(대각선) 수에서 +1, 글자가 동일하면 +0
            substitutions = matrix[current_row-1][j] + int(c1 != c2)

            # insertions, deletions, substitutions 중에서 가장 작은 값을 저장
            matrix[i+1][j + 1] = min(insertions, deletions, substitutions)
            # print(f"{insertions =} {deletions = } {substitutions = }")

        previous_row = current_row
    # for i in matrix:
        # print(i)
    return matrix[-1][-1] # 최종적으로 마지막 수가 결과


# 챗봇 클래스 정의
class SimpleChatBot:
    def __init__(self, filepath):
        self.questions, self.answers = self.load_data(filepath)

    def load_data(self, filepath):
        data = pd.read_csv(filepath)
        questions = data['Q'].tolist()
        answers = data['A'].tolist()
        return questions, answers

    # input_sentence와 유사도가 높은 questions에 대한 답변을 찾는 메서드
    def find_best_answer(self, input_sentence):
        # 가장 유사한 답변을 찾기 위해서 min_distance의 값을 가장 큰 값으로 초기화
        min_distance = float('inf')
        # 가장 적합한 답변의 index를 저장하기 위한 변수
        best_match_index = 0

        # input_sentence의 모든 questions에 대한 유사도를 측정
        for i, question in enumerate(self.questions):
            # levenshtein_distance함수를 사용해서 레벤슈타인 거리 계산
            distance = levenshtein_distance(input_sentence, question)
            if distance < min_distance: # min_distance보다 현재의 distance가 작으면
                min_distance = distance # 현재의 distance를 min_distance로 저장
                best_match_index = i # 현재의 index를 최적의 index로 저장
        return self.answers[best_match_index] # input_sentence와 유사도가 높은 questions의 index에 대한 답변을 반환

# 데이터 파일 경로
filepath = 'ChatbotData.csv'

# 챗봇 객체 생성
chatbot = SimpleChatBot(filepath)

# 사용자 입력 받기
while True:
    input_sentence = input('You: ')
    if input_sentence.lower() == '종료':
        break
    response = chatbot.find_best_answer(input_sentence)
    print('Chatbot:', response)


You: 안녕
Chatbot: 안녕하세요.
You: 제발 너 이제 된거니?
Chatbot: 다른 게임해보세요.
You: 악
Chatbot: 네
You: 려줘
Chatbot: 저도 즐거워요
You: 살려달라구
Chatbot: 방학이 참 짧죠.
You: 이게 내 최선이야
Chatbot: 아니에요. 너무 자책하지 마세요.
You: 고마워...
Chatbot: 감사합니다.
You: 너때문인데 사실..ㅠ
Chatbot: 딴 생각 하지마세요.
You: ㅠ 미안
Chatbot: 모두 제 잘못입니다.
You: 종료
